This notebook demonstrates how the `sample` method of `Model` objects can be used to efficiently calculate the fields at a set of points that are not arranged in a grid. The `calculate` method of `Model` objects automatically assembles a grid of sample points, calculates fields at all the grid points, then returns them in a `Results` object. The `Results` object has several methods for interpolating the results and is the best tool for examining results over the entire domain of a model. However, if an entire grid of results is not necessary, the `sample` method of `Model` objects provides an easy way to compute fields at an arbitrary set of points in the 3d space of the model. For example, it might be desireble to have a dense set of results along a fenceline so that the plotted results are nice and smooth. It might also be necessary to compute fields at a non-constant height. The grid of results created by the `calculate` method has a fixed height (3.28 ft by default). The `sample` method makes allows for different heights (z coordinates) because it computes fields at an arbitrary set of points in space.

In [1]:
import emf.subcalc as sc

Let's get straight to the sampling by loading a model from a tower template.

In [2]:
mod = sc.load_towers('towers.csv', return_model=True)

It doesn't really matter what we're modeling for this example, but the towers represent a single circuit running near a substation.

We can sample the `Model` object at any point in space that does not lie on the path of the model's wires (which causes divison by zero). Let's just say we want to know the fields at the point (1,2,3).

In [3]:
mod.sample(1,2,3)

segment calculations complete: 21/21 
total calculation time: 0.013 seconds


,,,Bmax,Bres,Bx,By,Bz,dist
x,y,z,,,,,,
1.0,2.0,3.0,0.131758,0.131805,0.076077,0.002704,0.107599,0.0


The `sample` method returns results in a pandas `DataFrame`. The returned frame has a `MultiIndex`, and is indexed by the sampled points. If we want just the maximum field at the point (1,2,3), we can get it.

In [4]:
print mod.sample(1,2,3).at[(1,2,3), 'Bmax']

segment calculations complete: 21/21 
total calculation time: 0.01 seconds
0.13175848903


It's also possible to omit the last argument to the `sample` method. In this case, the z coordinate is assumed to be the number stored in the `Model` object's `z` property. If the z coordinate is omitted, the returned frame is only indexed by the x and y coordinates.

In [5]:
mod.sample(1,2)

segment calculations complete: 21/21 
total calculation time: 0.015 seconds


,,Bmax,Bres,Bx,By,Bz,dist
x,y,,,,,,
1.0,2.0,0.13178,0.131826,0.076054,0.002673,0.107642,0.0


It's easy to get results for a single point, with or without a new z coordinate. Now we'll look at getting multiple points.

As a trivial example, let's get fields for the points (1,2,3), (4,5,6), and (7,8,9).

In [6]:
mod.sample([1,4,7], [2,5,8], [3,6,9])

segment calculations complete: 21/21 
total calculation time: 0.007 seconds


,,,Bmax,Bres,Bx,By,Bz,dist
x,y,z,,,,,,
1.0,2.0,3.0,0.131758,0.131805,0.076077,0.002704,0.107599,0.000000
4.0,5.0,6.0,0.133567,0.133615,0.076898,0.002435,0.109241,5.196152
7.0,8.0,9.0,0.135409,0.135457,0.077727,0.002276,0.110914,10.392305


Easy. Notice that the 'dist' column of the returned frame now has non-zero entries. That column shows the cumulative distance along the sampled points, from first to last. If the sampled points have no relation to one another, the 'dist' column can be ignored. If the sampled points represent a path through space, then the 'dist' column represents the distance along the path for each point, where the path is assumed to consist of straight lines between the points.

When sampling multiple points, the `sample` method can accept single coordinates or iterables for each argument. For example, if we wanted to sample the same points, but each with an x coordinate of 1, we could simply pass the number 1 to the x argument of `sample`, as below.

In [7]:
mod.sample(1, [2,5,8], [3,6,9])

segment calculations complete: 21/21 
total calculation time: 0.01 seconds


Bmax      Bres        Bx        By        Bz      dist
x   y   z                                                              
1.0 2.0 3.0  0.131758  0.131805  0.076077  0.002704  0.107599  0.000000
    5.0 6.0  0.132366  0.132413  0.076098  0.002279  0.108338  4.242641
    8.0 9.0  0.132971  0.133018  0.076106  0.002165  0.109074  8.485281

Any scalar arguments are applied to all sample points. We can do the same thing for the y coordinate.

In [8]:
mod.sample(1, 2, [3,6,9])

segment calculations complete: 21/21 
total calculation time: 0.015 seconds


Bmax      Bres        Bx        By        Bz  dist
x   y   z                                                          
1.0 2.0 3.0  0.131758  0.131805  0.076077  0.002704  0.107599   0.0
        6.0  0.131983  0.132030  0.075824  0.002409  0.108059   3.0
        9.0  0.132204  0.132251  0.075565  0.002221  0.108514   6.0

Neat!

The `sample` method has one final trick. By using the keyword argument `n`, we can direct the method to linearly interpolate between the input points and sample a total number of points roughly equal to the value passed to `n`. For example, imagine we want to sample along a path represented by the points (259,897), (910,728), and (970,377). However, we want to get about 100 samples along that path. We simply pass 100 to the keyword argument `n`.

In [9]:
points = [(259,897), (910,728), (970,377)]
x, y = zip(*points)
df = mod.sample(x, y, n=100)
print len(df)
print df.head(5)

segment calculations complete: 21/21 
total calculation time: 0.025 seconds
102
                           Bmax      Bres        Bx        By        Bz  \
x          y                                                              
259.000000 897.000000  0.178300  0.178303  0.153176  0.017596  0.089551   
268.863636 894.439394  0.185599  0.185601  0.159081  0.020628  0.093356   
278.727273 891.878788  0.193276  0.193278  0.165240  0.023957  0.097355   
288.590909 889.318182  0.201352  0.201353  0.171661  0.027609  0.101555   
298.454545 886.757576  0.209847  0.209848  0.178350  0.031612  0.105963   

                            dist  
x          y                      
259.000000 897.000000   0.000000  
268.863636 894.439394  10.190585  
278.727273 891.878788  20.381170  
288.590909 889.318182  30.571755  
298.454545 886.757576  40.762341  


The samples now include many points interpolated along the path of the input coordinates, for a total of 102 points. The sampled points will always include the input points, and we can get the results for those points by indexing the frame.

In [10]:
df.loc[points]

,,Bmax,Bres,Bx,By,Bz,dist
x,y,,,,,,
259.0,897.0,0.178300,0.178303,0.153176,0.017596,0.089551,0.00000
910.0,728.0,1.528188,1.528712,0.093798,0.943082,1.199483,672.57862
970.0,377.0,33.550348,34.014170,4.423154,9.526686,32.351843,1028.66990


In [11]:
#or we can get just the maximum fields for the input coordinates
df.loc[points].Bmax

x      y    
259.0  897.0     0.178300
910.0  728.0     1.528188
970.0  377.0    33.550348
Name: Bmax, dtype: float64

Using the `n` keyword enables dense sampling along a path of points. This can be useful if the results will be used for plotting because dense sampling generates smoother curves than interpolation of `Results` object's grid.